In [28]:
import os, sys

pysbf_path = "/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/"
sys.path.insert(0, pysbf_path)
from pysbf import *

In [29]:
import astropy.io.fits as fits
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab as py

In [30]:
def xcmd(cmd,verbose):

    if verbose: print('\n'+cmd)

    tmp=os.popen(cmd)
    output=''
    for x in tmp: output+=x
    if 'abort' in output:
        failure=True
    else:
        failure=tmp.close()
    if False:
        print('execution of %s failed' % cmd)
        print('error is as follows',output)
        sys.exit()
    else:
        return output
    
from contextlib import contextmanager
import os

@contextmanager
def cwd(path):
    oldpwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)
        
        
class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [31]:
def add_hdr(hdr, header, key):
    
    value = header[key]
    
    if isinstance(value, str):
        value = "'" + value + "'"
    else:
        value = str(value)
    
    hdr += key + "=" + value + " / " + header.comments[key] + "\n"
    
    return hdr

In [32]:
def getPGCid(objname):
    """extracting the PGC ID of a galaxy given its name

    Args:
        objname  (str): galaxy name

    Returns:
        int: PGC ID
    """

    
    my_url = 'http://ned.ipac.caltech.edu/cgi-bin/nph-objsearch?objname='+objname+'&extend=no&of=xml_names&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES'
    tag = mySoup(my_url).findAll('a', href="/cgi-bin/catdef?prefix=PGC")[0].parent
    pgc = int(str(tag).split("</a>")[1].split('</td>')[0].strip())
   
    return pgc

In [33]:
def addGalaxyInfo(Leda, pgc, response=None, objname=None):
    """Exctracting the information of a galaxy from the LEDA catalog 
    given the corresponsing PGC ID

    :param Leda: Leda catalog
    :type Leda: ``Pandas dataFrame``
    :param pgc: pgc ID
    :type pgc: ``int``
    :param response: response to be modified, defaults to None
    :type response: python ``dictionary``, optional
    :param objname: galaxy name, defaults to None
    :type objname: ``str``, optional
    :return: response dictionary that holds the requested galaxy information
    :rtype: python dictionary
    """

    if response is None:
        response = {}

    myGalaxy = Leda.loc[int(pgc)]
    response["pgc"]     = pgc
    response["ra"]      = "%.4f"%((myGalaxy.al2000)*15)         # deg
    response["dec"]     = "%.4f"%(myGalaxy.de2000)              # deg
    response["fov"]     = "%.2f"%(1.5*0.1*10**myGalaxy.logd25)  # arcmin
    response["pa"]      = str(myGalaxy.pa)

    if objname is None:
        response["objname"] = str(myGalaxy.objname)
    else:
        response["objname"] = objname

    return response

In [34]:
def loadLeda(leda_catalog):
    """Loading the HyperLeda catalog: 
    - http://leda.univ-lyon1.fr/
    - This catalog tabulates the proper information on local galaxies

    :return: the Leda catalog in the Pandas dataFrame format
    :rtype: Pandas ``dataFrame``
    """

    df = pd.read_csv(leda_catalog, delimiter=',')
    df = df.rename(columns=lambda x: x.strip())
                   
    return df.set_index('PGC')

In [35]:
Leda = loadLeda("LEDA.csv")

Leda.head()

/tmp/ipykernel_345233/963424436.py:10: DtypeWarning: Columns (71,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(leda_catalog, delimiter=',')


,al1950,de1950,al2000,de2000,l2,b2,sgl,sgb,t,e_t,...,objname,objtype,type,bar,ring,multiple,compactness,angclass,RA2000,DEC2000
PGC,,,,,,,,,,,,,,,,,,,,,
1,23.95695,0.42327,23.99966,0.70168,96.87946,-59.54180,293.35019,13.43002,2.4,5.0,...,PGC000001,G,S?,NaN,NaN,NaN,NaN,NaN,235958.8,4206.0
2,23.95786,46.99616,0.00047,47.27450,113.95535,-14.69916,341.64403,20.73880,3.1,0.4,...,UGC12889,G,Sb,B,NaN,NaN,NaN,NaN,1.7,471628.2
4,23.95829,22.80927,0.00096,23.08764,107.83219,-38.27289,316.05865,18.45140,5.0,1.2,...,PGC000004,G,Sc,NaN,NaN,NaN,NaN,NaN,3.5,230515.5
5,23.95988,32.46003,0.00254,32.73839,110.62060,-28.90431,326.17705,19.78055,-0.6,1.2,...,IC5370,G,S0-a,NaN,NaN,NaN,NaN,NaN,9.1,324418.2
6,23.95790,15.60327,0.00058,15.88165,105.25252,-45.21822,308.62424,17.10314,-1.0,2.0,...,PGC000006,G,S0-a,NaN,NaN,NaN,C,NaN,2.1,155253.9


In [9]:
# IC0511 - has z but missing g 

In [36]:
ned = get_ned_info(["eso352-g057"])

In [37]:
ned

,Row,Input Object Name,Object Name,RA,Dec,Gal_J2000.0,Gal_LON,Extinction,&_Heiles,PS1_g,PS1_r,PS1_i,PS1_z,PS1_y,Row,ra_dec
ESO352-G057,1,ESO352-G057,ESO 352- G 057,01h22m02.41s,-34d11m48.4s,261.874,-80.349,,0.088,0.063,0.047,0.037,0.030,0.065,1,"(20.510041666666663, -34.196777777777775)"


In [38]:
def load_info(galaxy_name, default_size = 0):
    
    ned = get_ned_info([galaxy_name])
    
    # import pdb; pdb.set_trace()
    
    try:
        if ned is not None:
            objName = ned.iloc[0]["Object Name"]
            ra, dec = ned.iloc[0]["ra_dec"]

            pgc = getPGCid(objName.replace(" ", "%20"))
            
            try:
                logd25 = Leda.loc[pgc]["logd25"]
            except:
                logd25 = default_size
            
            return pgc, objName.replace(" ", ""), ra, dec, logd25
    except: 
        print(f"Couldn't load data for {galaxy_name} ... !")
        pass
        
    return None

In [13]:
# example of available products for each brick

# https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/161/1610m107/

In [14]:
# ra = 194.3283 # 194.1825
# dec = 21.682

In [39]:
# d25 = 10**1.66
# pgc = 13318
# ra = 3.6076600  * 15.
# dec = -34.9761300 
# objNAME = 'NGC1380'


# d25 = 10**1.93
# pgc = 13418
# ra = 3.6414100 * 15.
# dec = -35.4506300
# objNAME = 'NGC1399'

d25 = 10**1.70
pgc = 13433
ra = 3.6477500  * 15.
dec = -35.5942400
objNAME = 'NGC1404'

In [40]:
bricks.head()

,BRICKNAME,BRICKID,BRICKQ,BRICKROW,BRICKCOL,RA,DEC,RA1,RA2,DEC1,DEC2
0,b'1800m900',1,1,0,0,180.0,-90.00,0.0,360.0,-90.000,-89.875
1,b'0180m897',2,2,1,0,18.0,-89.75,0.0,36.0,-89.875,-89.625
2,b'0540m897',3,3,1,1,54.0,-89.75,36.0,72.0,-89.875,-89.625
3,b'0900m897',4,2,1,2,90.0,-89.75,72.0,108.0,-89.875,-89.625
4,b'1260m897',5,3,1,3,126.0,-89.75,108.0,144.0,-89.875,-89.625


In [46]:
def get_decam(bricks, pgc, objName, ra, dec, logd25):

    temp_repo_name = "DECam/" + objName

    d25 = 10**logd25
    size = 0.1 * d25 / 60  # deg
    delta = size * 8
    pix_size = 0.262   # arcsec
    naxis = int(8*size*3600 / pix_size)
    
    if naxis > 5000:  # If the image is really big, just use max size of 5000 pix
        naxis=5000    # pixels for cutout
        delta = 0.365   # deg (5000 pix = 0.365 deg)
    if naxis < 1500 or d25==0:  # If the image is small, just use min size of 1500 pix
        naxis=1500    # pixels for cutout
        delta = 0.109   # deg (1500 pix = 0.109 deg)
    print('naxis = ', naxis)

    ra_max  =  ra+delta
    ra_min  =  ra-delta
    dec_max =  dec+delta
    dec_min =  dec-delta


    dg = bricks[(bricks["RA1"]<ra_max) & 
       (bricks["RA2"]>ra_min) &
       (bricks["DEC1"]<dec_max) & 
       (bricks["DEC2"]>dec_min) 
      ]



    if not os.path.isdir(temp_repo_name):
        os.mkdir(temp_repo_name)

    for filt in ["z"]:# ["g", "r", "i", "z"]:

        urls = [] 
        debugging_count = 0
        for iid in dg.BRICKNAME.values:
            
            debugging_count += 1

            s = str(iid)[2:-1]

            base_url = f"https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/{s[:3]}/{s}/"
            image_name = f"legacysurvey-{s}-image-{filt}.fits.fz"
            url = base_url + image_name

            print(url)

            urls.append(url)
            
#             if debugging_count==10:
#                 break
            

        with cwd(temp_repo_name):


            for url in urls:
                xcmd(f"wget {url}", verbose=True)


            cmd = 'mkdir tmp'
            xcmd(cmd, False)

            for url in urls:

                fname = url.split("/")[-1].split(".")[0]
                hdulist = fits.open(fname+".fits.fz")
                prihdr = hdulist[1].header 

                fits.writeto('tmp/' + fname+'.fits', hdulist[1].data, header=hdulist[1].header, 
                                        output_verify='exception', overwrite=True, checksum=False)


                fout=open('montage.csh','w')

            montage = '''
              rm *tbl
              rm -rf projected
              mkdir  projected
              mImgtbl tmp rimages.tbl
              mProjExec -p tmp rimages.tbl tmp.hdr projected stats.tbl
              mImgtbl projected pimages.tbl
              mAdd -p projected pimages.tbl tmp.hdr tmp.fits
              mConvert -b -32 tmp.fits tmp.fits
              rm *area*fits
              rm *tbl
              rm -rf projected
              '''

            fout.write(montage+'\n')
            fout.close()

            crpix = naxis/2
            hdr = ''
            hdr += 'SIMPLE  = T' + '\n'
            hdr += 'BITPIX  = -32' + '\n'
            hdr += 'NAXIS   = 2' + '\n'
            hdr += 'NAXIS1  = '+str(naxis) + '\n'
            hdr += 'NAXIS2  = '+str(naxis) + '\n'
            hdr += 'CTYPE1  = \'RA---TAN\'' + '\n'
            hdr += 'CTYPE2  = \'DEC--TAN\'' + '\n'
            hdr += 'CRVAL1  = ' +str(ra)+ '\n'
            hdr += 'CRVAL2  = ' +str(dec)+ '\n'
            hdr += 'CRPIX1  = '+str(crpix) + '\n'
            hdr += 'CRPIX2  = '+str(crpix) + '\n'
            hdr += 'CDELT1  = -7.27777778E-05' + '\n'
            hdr += 'CDELT2  = 7.27777778E-05' + '\n'
            hdr += 'CROTA2  = 0.000000' + '\n'
            hdr += 'EQUINOX = 2000.0' + '\n'
            hdr += 'BSCALE  = 1' + '\n'
            hdr += 'BZERO   = 0' + '\n'
            hdr += 'EXPTIME = 1.0' + '\n'
#             hdr += 'ZP = 16.40006562 ' + '\n'
            hdr = add_hdr(hdr, prihdr, "MAGZERO")
            hdr = add_hdr(hdr, prihdr, "BUNIT")
            hdr = add_hdr(hdr, prihdr, "COSKY_"+filt.upper())
            hdr = add_hdr(hdr, prihdr, "FILTER")
            hdr = add_hdr(hdr, prihdr, "FILTERX")
            hdr = add_hdr(hdr, prihdr, "INSTRUME")
            hdr = add_hdr(hdr, prihdr, "TELESCOP")
            hdr = add_hdr(hdr, prihdr, "OBSERVAT")
            hdr = add_hdr(hdr, prihdr, "EQUINOX")
            hdr = add_hdr(hdr, prihdr, "LSDR")
            hdr = add_hdr(hdr, prihdr, "SURVEY")
            hdr = add_hdr(hdr, prihdr, "SURVEYID")
            hdr = add_hdr(hdr, prihdr, "LEGPIPEV")
        #     hdr = add_hdr(hdr, prihdr, "")
        #     hdr = add_hdr(hdr, prihdr, "")

            hdr += 'PGC = ' + 'PGC'+str(pgc) + '\n'
            hdr += 'OBJECT = ' + objName + '\n'
            hdr += 'END' + '\n'

            fout=open('tmp.hdr','w')
            fout.write(hdr)
            fout.close()
            
            
            print("done ...")
            # break

            xcmd('csh montage.csh', False)
            xcmd('rm montage.csh', False)
            xcmd('rm tmp.hdr', False)
            xcmd('rm -rf tmp', False)
            xcmd('rm -rf *.fits.fz*', False)
            xcmd(f'mv tmp.fits {objName}_{filt}.fits', True)

In [47]:
from astropy.table import Table

data = Table.read('survey-bricks.fits', format='fits')
bricks = data.to_pandas()

In [48]:

galaxies = [
# "IC3025",
# "IC3032",
# "IC3487",
# "IC3586",
# "N4458",
#"N4472",
#"N4489",
## "N4649",

# "eso352-g057", # didn't work, Should have data
# "n4125",

"n6841",
    
#"cgcg-005-038",  # works
#"cgcg-031-049"  # doesn't work
]


In [49]:
# for galaxy in ["IC1919", "IC2006", "N1344", "N1374", "N1375"]:
for galaxy in galaxies:

    pgc, objName, ra, dec, logd25 = load_info(galaxy)    
    get_decam(bricks, pgc, galaxy, ra, dec, logd25)

naxis =  2971
https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2992m320/legacysurvey-2992m320-image-z.fits.fz
https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2995m320/legacysurvey-2995m320-image-z.fits.fz
https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2991m317/legacysurvey-2991m317-image-z.fits.fz
https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2994m317/legacysurvey-2994m317-image-z.fits.fz
https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2997m317/legacysurvey-2997m317-image-z.fits.fz
https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2992m315/legacysurvey-2992m315-image-z.fits.fz
https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2995m315/legacysurvey-2995m315-image-z.fits.fz

wget https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2992m320/legacysurvey-2992m320-image-z.fits.fz


--2024-07-24 07:41:54--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2992m320/legacysurvey-2992m320-image-z.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.106
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.106|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-07-24 07:41:57 ERROR 404: Not Found.

--2024-07-24 07:41:57--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2995m320/legacysurvey-2995m320-image-z.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.106
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.106|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-07-24 07:41:57 ERROR 404: Not Found.

--2024-07-24 07:41:57--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2991m317/legacysurvey-2991m317-image-z.fits.fz



wget https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2995m320/legacysurvey-2995m320-image-z.fits.fz

wget https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2991m317/legacysurvey-2991m317-image-z.fits.fz


Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.106
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.106|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-07-24 07:41:57 ERROR 404: Not Found.

--2024-07-24 07:41:57--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2994m317/legacysurvey-2994m317-image-z.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.106
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.106|:443... connected.



wget https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2994m317/legacysurvey-2994m317-image-z.fits.fz

wget https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2997m317/legacysurvey-2997m317-image-z.fits.fz


HTTP request sent, awaiting response... 404 Not Found
2024-07-24 07:41:58 ERROR 404: Not Found.

--2024-07-24 07:41:58--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2997m317/legacysurvey-2997m317-image-z.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.106
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.106|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-07-24 07:41:58 ERROR 404: Not Found.

--2024-07-24 07:41:58--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2992m315/legacysurvey-2992m315-image-z.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.106
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.106|:443... connected.
HTTP request sent, awaiting response... 


wget https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2992m315/legacysurvey-2992m315-image-z.fits.fz


404 Not Found
2024-07-24 07:41:59 ERROR 404: Not Found.

--2024-07-24 07:41:59--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2995m315/legacysurvey-2995m315-image-z.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.106
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.106|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-07-24 07:41:59 ERROR 404: Not Found.




wget https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2995m315/legacysurvey-2995m315-image-z.fits.fz


FileNotFoundError: [Errno 2] No such file or directory: 'legacysurvey-2992m320-image-z.fits.fz'

In [ ]:
pgc, objName, ra, dec, logd25

In [ ]:
def has_decam(bricks, galaxy_name):

    pgc, objName, ra, dec, logd25 = load_info(galaxy_name)

    d25 = 10**logd25
    size = 0.1 * d25 / 60  # deg
    delta = size * 8
    pix_size = 0.262   # arcsec
    naxis = int(6*size*3600 / pix_size)

    ra_max  =  ra+delta
    ra_min  =  ra-delta
    dec_max =  dec+delta
    dec_min =  dec-delta


    dg = bricks[(bricks["RA1"]<ra_max) & 
       (bricks["RA2"]>ra_min) &
       (bricks["DEC1"]<dec_max) & 
       (bricks["DEC2"]>dec_min) 
      ]
    
    
    return dg.shape[0]


In [ ]:
has_decam(bricks, "NGC4649")